['id','click','hour','banner_pos','device_id','device_ip','device_model','device_conn_type','C14','C17','C20','C21'], and add cols ['pub_id','pub_domain','pub_category','device_id_count','device_ip_count','user_count','smooth_user_hour_count','user_click_histroy'].

Stats:
- 'small' with device_* cols: Private=0.3998675, Public=0.4020308
- 'small' without device_* cols: Private=0.4024273, Public=0.4045937


TODO:
- use a faster df_to_ffm for the training set. Should fit in memory.
- count features: device_id, device_ip, user

In [1]:
import xlearn as xl

from tools.ffm_tools import make_train_validate_data, df_to_ffm, write_submission
from models.base import create_user, site_app_split

In [2]:
import subprocess

def xlearn_train_command(train_set, validation_set, model_type='2', learning_rate=None, reg_param=None):
    xlearn_train = '~/code/xlearn/build/xlearn_train'
    command = [xlearn_train, train_set,
          '-s', str(model_type),
         '-v', validation_set]
    if learning_rate:
        command.append('-r')
        command.append(str(learning_rate))                      
    if reg_param:
        command.append('-b')
        command.append('{0:.7f}'.format(reg_param))
    return ' '.join(command)

def xlearn_predict_command(test_set, model):
    xlearn_predict = '~/code/xlearn/build/xlearn_predict'
    command = [xlearn_predict, test_set, model, '--sigmoid']
    return ' '.join(command)

## With count features
- device_ip, device_id, user
- hourly user

Stats:
- 'small': with device_* cols. 01m45s for site (spilled a tiny bit--a few MB--to swap), 01m for app.
    Validation scores: site=0.447193, app=0.340356 (weighted average = 0.3979).
    Private 0.3974360, Public 0.3993231.
- 'small': Without device_* cols. 01s with fast=True to prepare train sets (only about 300M memory consumption). 04m25s if fast=False.
    01m20s for test_site, 50s for test_app, also fast=True. Memory consumption not an issue.
- 'small', with device_* cols. 01m24s for train.
- 'small'. site: (lr, reg) =  (0.0725, 0.0000213), app: (0.149, 0.0000151) Private 0.3983551, Public 0.4003566


In [23]:
a = 48252
s = 56347
0.447193 * (s/(a+s)) + 0.340356 * (a/(a+s))

0.3979086003021062

In [3]:
import pandas as pd

# categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
#                     'user', 'device_conn_type', 'C14','C17','C20','C21',
#                        'device_id_count', 'device_ip_count', 'user_count']
#  add device_id, device_ip, and device_model.
categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                       'device_id', 'device_ip', 'device_model',
                   'user', 'device_conn_type', 'C14','C17','C20','C21',
                       'device_id_count', 'device_ip_count', 'user_count']

train_size='small'

df = pd.read_csv(f'./data/train_{train_size}.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)

In [6]:
def add_count_col(df, col, new_col_name):
    counts = df.groupby(col).size().to_frame(new_col_name)
    df = pd.merge(df, counts, how='left', on=col)
    df = df.fillna({new_col_name: 0})
    return df, counts

In [7]:
def merge_counts(df, counts, col, count_col):
    df = pd.merge(df, counts, how='left', on=col)
    df[count_col] = df[count_col].fillna(0).astype(int)
    return df

In [8]:
df, df_device_ip_count = add_count_col(df, 'device_ip', 'device_ip_count')
df, df_device_id_count = add_count_col(df, 'device_id', 'device_id_count')
df, df_user_count = add_count_col(df, 'user', 'user_count')
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C17,C18,C19,C20,C21,device_ip_model,user,device_ip_count,device_id_count,user_count
0,11761126300016035614,0,2014-10-26 06:00:00,1002,0,6b175d24,a1b0ebd0,50e219e0,ecad2386,7801e8d9,...,1722,0,35,-1,79,0d34e1f4_52c5cc40,54eeaef3,1,1,1
1,9454951464027616243,0,2014-10-26 06:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1722,0,35,-1,79,fc61627b_711ee120,fc61627b_711ee120,13,824934,4
2,7597529435648422906,0,2014-10-24 07:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,92f5800b,ae637522,...,2424,1,161,100193,71,68b6ba25_981edffc,68b6ba25_981edffc,12,824934,6
3,17674252474717503445,0,2014-10-26 16:00:00,1005,0,2b1ddb24,98acf46c,3e814130,ecad2386,7801e8d9,...,1973,3,39,100148,23,75092a71_ecb851b2,75092a71_ecb851b2,1,824934,1
4,14668325104229564251,0,2014-10-25 04:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1722,0,35,100083,79,5bb6c0b5_8a4875bd,5bb6c0b5_8a4875bd,30,824934,4


In [9]:
df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C17,C18,C19,C20,C21,device_ip_model,user,device_ip_count,device_id_count,user_count
0,11761126300016035614,0,2014-10-26 06:00:00,1002,0,6b175d24,a1b0ebd0,50e219e0,ecad2386,7801e8d9,...,1722,0,35,-1,79,0d34e1f4_52c5cc40,54eeaef3,1,1,1
1,9454951464027616243,0,2014-10-26 06:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1722,0,35,-1,79,fc61627b_711ee120,fc61627b_711ee120,13,824934,4
2,7597529435648422906,0,2014-10-24 07:00:00,1005,0,85f751fd,c4e18dd6,50e219e0,92f5800b,ae637522,...,2424,1,161,100193,71,68b6ba25_981edffc,68b6ba25_981edffc,12,824934,6
3,17674252474717503445,0,2014-10-26 16:00:00,1005,0,2b1ddb24,98acf46c,3e814130,ecad2386,7801e8d9,...,1973,3,39,100148,23,75092a71_ecb851b2,75092a71_ecb851b2,1,824934,1
4,14668325104229564251,0,2014-10-25 04:00:00,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1722,0,35,100083,79,5bb6c0b5_8a4875bd,5bb6c0b5_8a4875bd,30,824934,4


In [10]:
%%time
# we don't use train_site/app, validate_site/app
fast=True
_, _, feature_dict_site, _, _, feature_dict_app = make_train_validate_data(df, categorical_features, train_size, fast=fast)

./ffm-data/train_site_small.ffm


/home/atkm/code/avazu-ctr/tools/ffm_tools.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[c] = df[c].map(replace_dict).fillna(feature_dict_size).astype(int)
/home/atkm/code/avazu-ctr/tools/ffm_tools.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = f'{i}:' + df[col].astype('str') + ':1'


./ffm-data/validate_site_small.ffm
./ffm-data/train_app_small.ffm
./ffm-data/validate_app_small.ffm
CPU times: user 1min 22s, sys: 2.12 s, total: 1min 24s
Wall time: 1min 24s


In [11]:
del df

In [12]:
train_set = f'./ffm-data/train_site_{train_size}.ffm'
validation_set = f'./ffm-data/validate_site_{train_size}.ffm'

#(lr, reg) = (0.0725, 0.0000213) # w/o device_* cols
#(lr, reg) = (0.105, 0.00000130) # w/o device_* cols
(lr, reg) = (0.137, 0.00000779) # w/ device_* cols
run_training_site = xlearn_train_command(train_set, validation_set, learning_rate=lr, reg_param=reg)
print(run_training_site)
result = subprocess.run(run_training_site, stdout=subprocess.PIPE, shell=True)
result.stdout.decode()

~/code/xlearn/build/xlearn_train ./ffm-data/train_site_small.ffm -s 2 -v ./ffm-data/validate_site_small.ffm -r 0.137 -b 0.0000078


"\x1b\x1b----------------------------------------------------------------------------------------------\n           _\n          | |\n     __  _| |     ___  __ _ _ __ _ __\n     \\ \\/ / |    / _ \\/ _` | '__| '_ \\ \n      >  <| |___|  __/ (_| | |  | | | |\n     /_/\\_\\_____/\\___|\\__,_|_|  |_| |_|\n\n        xLearn   -- 0.35 Version --\n----------------------------------------------------------------------------------------------\n\n\x1b\x1b\x1b\x1b[ ACTION     ] Read Problem ...\x1b\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/train_site_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/validate_site_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bNumber of Feature: 840427\n\x1b[------------] \x1bNumber o

In [14]:
train_set = f'./ffm-data/train_app_{train_size}.ffm'
validation_set = f'./ffm-data/validate_app_{train_size}.ffm'
#(lr, reg) = (0.149, 0.0000151) # w/o device_* cols
#(lr, reg) = (0.384, 0.0000921) # w/o device_* cols
(lr, reg) = (0.165, 0.00006) # w/ device_* cols

run_training_app = xlearn_train_command(train_set, validation_set, learning_rate=lr, reg_param=reg)
print(run_training_app)
result = subprocess.run(run_training_app, stdout=subprocess.PIPE, shell=True)
result.stdout.decode()

~/code/xlearn/build/xlearn_train ./ffm-data/train_app_small.ffm -s 2 -v ./ffm-data/validate_app_small.ffm -r 0.165 -b 0.0000600


"\x1b\x1b----------------------------------------------------------------------------------------------\n           _\n          | |\n     __  _| |     ___  __ _ _ __ _ __\n     \\ \\/ / |    / _ \\/ _` | '__| '_ \\ \n      >  <| |___|  __/ (_| | |  | | | |\n     /_/\\_\\_____/\\___|\\__,_|_|  |_| |_|\n\n        xLearn   -- 0.35 Version --\n----------------------------------------------------------------------------------------------\n\n\x1b\x1b\x1b\x1b[ ACTION     ] Read Problem ...\x1b\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/train_app_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/validate_app_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bNumber of Feature: 533535\n\x1b[------------] \x1bNumber of 

In [15]:
test_size='full'

if test_size == 'full':
    df_test = pd.read_csv(f'./data/test.csv', dtype={'id': 'uint64'})
else:
    df_test = pd.read_csv(f'./data/test_{test_size}.csv', dtype={'id': 'uint64'})

df_test = create_user(df_test)
df_test = merge_counts(df_test, df_device_ip_count, 'device_ip', 'device_ip_count')
df_test = merge_counts(df_test, df_device_id_count, 'device_id', 'device_id_count')
df_test = merge_counts(df_test, df_user_count, 'user', 'user_count')

df_test_site, df_test_app = site_app_split(df_test)
del df_test
df_test_site.shape, df_test_app.shape

((2858160, 25), (1719304, 25))

In [16]:
%%time
ffm_data_test_site = f'./ffm-data/test_site_{test_size}.ffm'
    
fast=True
df_to_ffm(df_test_site, categorical_features, 
          ffm_data_test_site, 'test', feature_dict_site, fast)

/home/atkm/code/avazu-ctr/tools/ffm_tools.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['dummy'] = np.ones(len(df))
/home/atkm/code/avazu-ctr/tools/ffm_tools.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['dummy'] = df['dummy'].astype(int)


CPU times: user 1min 39s, sys: 5.67 s, total: 1min 45s
Wall time: 1min 45s


In [17]:
%%time
ffm_data_test_app = f'./ffm-data/test_app_{test_size}.ffm'

df_to_ffm(df_test_app, categorical_features, 
          ffm_data_test_app, 'test', feature_dict_app, fast)

CPU times: user 59.2 s, sys: 3.63 s, total: 1min 2s
Wall time: 1min 3s


In [ ]:
raise

## Expedite data generation

In [2]:
import pandas as pd

train_size='tiny'

categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                    'user', 'device_conn_type', 'C14','C17','C20','C21']


df = pd.read_csv(f'./data/train_{train_size}.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)

In [3]:
from tools.cv_tools import train_test_split

test_day = 30
df_train, df_validate = train_test_split(df, None, test_day)
df_train_site, df_train_app = site_app_split(df_train)
df_validate_site, df_validate_app = site_app_split(df_validate)

In [9]:
from tools.ffm_tools import df_to_ffm

df_to_ffm(df_validate_site, categorical_features, 'test-write.csv', 'validate', feature_dict_site, fast=True)

/home/atkm/code/avazu-ctr/tools/ffm_tools.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[c] = df[c].map(replace_dict).fillna(feature_dict_size).astype(int)
/home/atkm/code/avazu-ctr/tools/ffm_tools.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = f'{i}:' + df[col].astype('str') + ':1'


In [4]:
from tools.ffm_tools import make_feature_dict, encode_features

feature_dict_site = make_feature_dict(df_train_site, categorical_features)

In [8]:
encoded = encode_features(df_train_site, feature_dict_site, categorical_features)

In [9]:
encoded.head()

,id,click,hour,C1,banner_pos,platform_id,platform_domain,platform_category,device_id,device_ip,...,C14,C15,C16,C17,C18,C19,C20,C21,device_ip_model,user
0,11066293153052526398,0,2014-10-23 02:00:00,1005,0,2,152,274,a99f214a,4665f0a5,...,871,300,250,1095,0,35,1214,1259,4665f0a5_36d749e5,283
1,8222349973801650597,1,2014-10-22 10:00:00,1005,0,2,152,274,a99f214a,80df79b5,...,872,300,250,1096,2,35,1215,1260,80df79b5_76dc4769,284
2,9513665962503843628,0,2014-10-21 17:00:00,1005,0,3,153,275,a99f214a,693bff3e,...,873,320,50,1097,0,35,1214,1259,693bff3e_4ea23a13,285
3,7946799301127038469,0,2014-10-24 08:00:00,1005,0,4,154,276,a99f214a,2773b1c8,...,874,320,50,1098,3,39,1214,1260,2773b1c8_d787e91b,286
4,13112131110372814102,0,2014-10-22 19:00:00,1005,1,5,155,276,a99f214a,b200f41e,...,875,320,50,1099,0,35,1216,1259,b200f41e_8a4875bd,287


In [21]:
validate_encoded = encode_features(df_validate_site[categorical_features], feature_dict_site, categorical_features)

/home/atkm/code/avazu-ctr/tools/ffm_tools.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[c] = df[c].map(replace_dict)


In [31]:
feature_dict_site.__missing__ = lambda : 0

In [36]:
d = {'seen': 0}

In [38]:
d.__missing__ = lambda : 0

AttributeError: 'dict' object has no attribute '__missing__'

In [37]:
pd.DataFrame({'c': ['unseen']}).c.map(d)

0   NaN
Name: c, dtype: float64

In [22]:
validate_encoded.head()

,banner_pos,platform_id,platform_domain,platform_category,user,device_conn_type,C14,C17,C20,C21
14,1,53.0,199.0,276,NaN,869,896.0,1113.0,1215.0,1269.0
50,0,NaN,NaN,277,NaN,869,1003.0,1172.0,1223.0,1260.0
58,0,3.0,153.0,275,NaN,869,1017.0,1181.0,1214.0,1281.0
67,0,NaN,NaN,274,NaN,869,NaN,1197.0,1218.0,1267.0
124,0,NaN,NaN,274,NaN,869,976.0,1159.0,1214.0,1260.0


In [ ]:
encoded[categorical_features ].isna().sum()

In [ ]:
encoded = encoded[categorical_features + ['click']]

In [ ]:
encoded.head()

In [ ]:
copied = encoded.copy()

In [ ]:
copied.banner_pos = '0:' + copied.banner_pos.astype('str') + ':1'

In [ ]:
for i, c in enumerate(copied):
    copied[c] = f'{i}:' + copied[c].astype('str') + ':1'

In [ ]:
copied.info(memory_usage='deep')

In [ ]:
copied.head()

In [ ]:
encoded.info(memory_usage='deep')

In [ ]:
copied.shape

In [ ]:
%%time
copied.to_csv('test-write-encoded.csv', sep=' ')

In [ ]:
encoded.shape

In [ ]:
%%time
from tools.ffm_tools import ffm_row_generator
with open('test-write-encoded.ffm', 'w') as f:
    for ffm_row in ffm_row_generator(encoded):
        f.write(ffm_row)
        f.write('\n')

In [ ]:
raise

## Without count features

### Part 0: prepare train data.

Run `~/code/xlearn/build/xlearn_train ./ffm-data/train_{site,app}_{train_size}.ffm -s 2 -v ./ffm-data/validate_{site,app}_{train_size}.ffm`.

Generates `./ffm-data/train_{site,app}_{train_size}.ffm.model`.

Early-stopping is enabled by default when a validation set is provided.

Stats:
- 'small', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category', 'user', 'device_conn_type', 'C14','C17','C20','C21'] => 04m25s (with the new ffm_row_generator).

    Use (lr, reg) = (1.5055668655636434, 0.14551999446480063) for site; (0.00166189386987065, 0.48305136367563084) for app.
    Private=0.4278890, Public=0.4292961
    
    Use site: (lr, reg) =  (0.062, 0.00000561), app: (0.433, 0.000104).
    Private=0.3977579, Public=0.3997753
    
- 'small', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
    'device_id', 'device_ip', 'device_model',
    'user', 'device_conn_type', 'C14','C17','C20','C21'] => 09m15s, best losses: site-0.447526, app-0.343323.

In [6]:
import pandas as pd

categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
                    'user', 'device_conn_type', 'C14','C17','C20','C21']
#  add device_id, device_ip, and device_model.
#categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
#                        'device_id', 'device_ip', 'device_model',
#                    'user', 'device_conn_type', 'C14','C17','C20','C21']

train_size='small'

df = pd.read_csv(f'./data/train_{train_size}.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
df = create_user(df)

In [7]:
%%time
# we don't use train_site/app, validate_site/app
_, _, feature_dict_site, _, _, feature_dict_app = make_train_validate_data(df, categorical_features, train_size)

./ffm-data/train_site_small.ffm
./ffm-data/validate_site_small.ffm
./ffm-data/train_app_small.ffm
./ffm-data/validate_app_small.ffm
CPU times: user 4min 54s, sys: 1.6 s, total: 4min 56s
Wall time: 5min


In [ ]:
raise "Tune Model"

In [8]:
train_set = f'./ffm-data/train_site_{train_size}.ffm'
validation_set = f'./ffm-data/validate_site_{train_size}.ffm'

(lr, reg) = (0.062, 0.00000561)
run_training_site = xlearn_train_command(train_set, validation_set, learning_rate=lr, reg_param=reg)
run_training_site

'~/code/xlearn/build/xlearn_train ./ffm-data/train_site_small.ffm -s 2 -v ./ffm-data/validate_site_small.ffm -r 0.062 -b 5.61e-06'

In [9]:
result = subprocess.run(run_training_site, stdout=subprocess.PIPE, shell=True)
result.stdout.decode()

"\x1b\x1b----------------------------------------------------------------------------------------------\n           _\n          | |\n     __  _| |     ___  __ _ _ __ _ __\n     \\ \\/ / |    / _ \\/ _` | '__| '_ \\ \n      >  <| |___|  __/ (_| | |  | | | |\n     /_/\\_\\_____/\\___|\\__,_|_|  |_| |_|\n\n        xLearn   -- 0.35 Version --\n----------------------------------------------------------------------------------------------\n\n\x1b\x1b\x1b\x1b[ ACTION     ] Read Problem ...\x1b\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/train_site_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/validate_site_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bNumber of Feature: 467850\n\x1b[------------] \x1bNumber o

In [10]:
train_set = f'./ffm-data/train_app_{train_size}.ffm'
validation_set = f'./ffm-data/validate_app_{train_size}.ffm'
(lr, reg) = (0.433, 0.000104)

run_training_app = xlearn_train_command(train_set, validation_set, learning_rate=lr, reg_param=reg)
run_training_app

'~/code/xlearn/build/xlearn_train ./ffm-data/train_app_small.ffm -s 2 -v ./ffm-data/validate_app_small.ffm -r 0.433 -b 0.000104'

In [11]:
result = subprocess.run(run_training_app, stdout=subprocess.PIPE, shell=True)
result.stdout.decode()

"\x1b\x1b----------------------------------------------------------------------------------------------\n           _\n          | |\n     __  _| |     ___  __ _ _ __ _ __\n     \\ \\/ / |    / _ \\/ _` | '__| '_ \\ \n      >  <| |___|  __/ (_| | |  | | | |\n     /_/\\_\\_____/\\___|\\__,_|_|  |_| |_|\n\n        xLearn   -- 0.35 Version --\n----------------------------------------------------------------------------------------------\n\n\x1b\x1b\x1b\x1b[ ACTION     ] Read Problem ...\x1b\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/train_app_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary format.\n\x1b[------------] \x1bBinary file (./ffm-data/validate_app_small.ffm.bin) found. Skip converting text to binary.\n\x1b[------------] \x1bNumber of Feature: 241763\n\x1b[------------] \x1bNumber of 

In [ ]:
raise "Train/tune models."

### Part 1: prepare test data.

Generates `./ffm-data/test_{site,app}_{test_size}.ffm.out`.

Stats:
- 'full', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category', 'user', 'device_conn_type', 'C14','C17','C20','C21']  --> site = 08m42s (old 27m); app = 05m19s (old 14m).
- 'small', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category', 'user', 'device_conn_type', 'C14','C17','C20','C21']  --> site = 01m53s (old 06m); app = 01m09s (old 4m).
- 'full', with categorical_features = ['banner_pos', 'platform_id', 'platform_domain', 'platform_category',
    'device_id', 'device_ip', 'device_model',
    'user', 'device_conn_type', 'C14','C17','C20','C21'] => site = 25m39s; app = 15m45s.

In [5]:
test_size='tiny'

if test_size == 'full':
    df_test = pd.read_csv(f'./data/test.csv', dtype={'id': 'uint64'})
else:
    df_test = pd.read_csv(f'./data/test_{test_size}.csv', dtype={'id': 'uint64'})
df_test = create_user(df_test)

df_test_site, df_test_app = site_app_split(df_test)
del df_test
df_test_site.shape, df_test_app.shape

((612, 22), (388, 22))

In [6]:
%%time
ffm_data_test_site = f'./ffm-data/test_site_{test_size}.ffm'
    
df_to_ffm(df_test_site, categorical_features, 
          ffm_data_test_site, 'test', feature_dict_site, fast=True)

/home/atkm/code/avazu-ctr/tools/ffm_tools.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[c] = df[c].map(replace_dict).fillna(feature_dict_size).astype(int)
/home/atkm/code/avazu-ctr/tools/ffm_tools.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = f'{i}:' + df[col].astype('str') + ':1'


CPU times: user 867 ms, sys: 7.8 ms, total: 875 ms
Wall time: 873 ms


/home/atkm/code/avazu-ctr/tools/ffm_tools.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['dummy'] = np.ones(len(df))
/home/atkm/code/avazu-ctr/tools/ffm_tools.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['dummy'] = df['dummy'].astype(int)


In [14]:
%%time
ffm_data_test_app = f'./ffm-data/test_app_{test_size}.ffm'

df_to_ffm(df_test_app, categorical_features, 
          ffm_data_test_app, 'test', feature_dict_app)

CPU times: user 5min 38s, sys: 1.63 s, total: 5min 40s
Wall time: 5min 41s


### Part 2: predict

In [18]:
test_set = f'./ffm-data/test_site_{test_size}.ffm'
model = f'./ffm-data/train_site_{train_size}.ffm.model'
run_predict_site = xlearn_predict_command(test_set, model)
run_predict_site

'~/code/xlearn/build/xlearn_predict ./ffm-data/test_site_full.ffm ./ffm-data/train_site_small.ffm.model --sigmoid'

In [19]:
result = subprocess.run(run_predict_site, stdout=subprocess.PIPE, shell=True)
result.stdout.decode()

"\x1b\x1b----------------------------------------------------------------------------------------------\n           _\n          | |\n     __  _| |     ___  __ _ _ __ _ __\n     \\ \\/ / |    / _ \\/ _` | '__| '_ \\ \n      >  <| |___|  __/ (_| | |  | | | |\n     /_/\\_\\_____/\\___|\\__,_|_|  |_| |_|\n\n        xLearn   -- 0.35 Version --\n----------------------------------------------------------------------------------------------\n\n\x1b\x1b\x1b\x1b[ ACTION     ] Load model ...\x1b\n\x1b[------------] \x1bLoad model from ./ffm-data/train_site_small.ffm.model\n\x1b[------------] \x1bLoss function: cross-entropy\n\x1b[------------] \x1bScore function: ffm\n\x1b[------------] \x1bNumber of Feature: 840427\n\x1b[------------] \x1bNumber of K: 4\n\x1b[------------] \x1bNumber of field: 16\n\x1b[------------] \x1bTime cost for loading model: 0.89 (sec)\n\x1b\x1b[ ACTION     ] Read Problem ...\x1b\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary fo

In [20]:
test_set = f'./ffm-data/test_app_{test_size}.ffm'
model = f'./ffm-data/train_app_{train_size}.ffm.model'
run_predict_app = xlearn_predict_command(test_set, model)
run_predict_app

'~/code/xlearn/build/xlearn_predict ./ffm-data/test_app_full.ffm ./ffm-data/train_app_small.ffm.model --sigmoid'

In [21]:
result = subprocess.run(run_predict_app, stdout=subprocess.PIPE, shell=True)
result.stdout.decode()

"\x1b\x1b----------------------------------------------------------------------------------------------\n           _\n          | |\n     __  _| |     ___  __ _ _ __ _ __\n     \\ \\/ / |    / _ \\/ _` | '__| '_ \\ \n      >  <| |___|  __/ (_| | |  | | | |\n     /_/\\_\\_____/\\___|\\__,_|_|  |_| |_|\n\n        xLearn   -- 0.35 Version --\n----------------------------------------------------------------------------------------------\n\n\x1b\x1b\x1b\x1b[ ACTION     ] Load model ...\x1b\n\x1b[------------] \x1bLoad model from ./ffm-data/train_app_small.ffm.model\n\x1b[------------] \x1bLoss function: cross-entropy\n\x1b[------------] \x1bScore function: ffm\n\x1b[------------] \x1bNumber of Feature: 533535\n\x1b[------------] \x1bNumber of K: 4\n\x1b[------------] \x1bNumber of field: 16\n\x1b[------------] \x1bTime cost for loading model: 0.56 (sec)\n\x1b\x1b[ ACTION     ] Read Problem ...\x1b\n\x1b[------------] \x1bFirst check if the text file has been already converted to binary for

In [ ]:
raise "run prediction"

### Part 3: write submission.csv

In [22]:
prediction_site = f'./ffm-data/test_site_{test_size}.ffm'
prediction_site_out = prediction_site + '.out'
prediction_site_id = prediction_site + '.id'
prediction_app = f'./ffm-data/test_app_{test_size}.ffm'
prediction_app_out = prediction_app + '.out'
prediction_app_id = prediction_app + '.id'

write_submission(prediction_site_out, prediction_site_id, prediction_app_out, prediction_app_id, 'submission-.csv')

### Part 2 (obsolete): train model and predict.
Do this in shell.

In [ ]:
def run_ffm(train_set, validate_set, test_set, model_out, predict_out):
    ffm_model = xl.create_ffm()
    ffm_model.setTrain(train_set)
    ffm_model.setValidate(validate_set)
    
    param = {'task':'binary', 'lr':0.2, 'lambda':0.002}
    ffm_model.fit(param, model_out)
    
    ffm_model.setSigmoid()
    ffm_model.setTest(test_set)
    ffm_model.predict(model_out, predict_out)
    print(f'Prediction written to {predict_out}')

In [ ]:
#TODO: move this to tools/kaggle_tools
def validate_submission(submission_csv, ids):
    """
    Ensure that the submission file is in the right format, 
    and that it contains required id's.
    """
    pass

In [ ]:
train_size = 'small'
test_size = 'small'

In [ ]:
train_site = f'./ffm-data/train_site_{train_size}.ffm'
validate_site = f'./ffm-data/validate_site_{train_size}.ffm'
test_site = f'./ffm-data/test_site_{test_size}.ffm'
model_site_out = './xlearn-out/ffm-site.out'
predict_site_out = './xlearn-out/out-site.txt'

In [ ]:
%%time
# TODO: kernel dies when the test 
run_ffm(train_site, validate_site, test_site, model_site_out, predict_site_out)

In [ ]:
train_app = f'./ffm-data/train_app_{train_size}.ffm'
validate_app = f'./ffm-data/validate_app_{train_size}.ffm'
test_app = f'./ffm-data/test_app_{test_size}.ffm'
model_app_out = './xlearn-out/ffm-app.out'
predict_app_out = './xlearn-out/out-app.txt'

run_ffm(train_app, validate_app, test_app, model_app_out, predict_app_out)

In [ ]:
prediction_id_site = f'./ffm-data/test_site_{test_size}.ffm.id'
prediction_id_app = f'./ffm-data/test_app_{test_size}.ffm.id'
write_submission(predict_site_out, prediction_id_site, predict_app_out, prediction_id_app, 'submission-.csv')